In [48]:
import pandas as pd
import json
import altair as alt


In [14]:

  
# Opening JSON file
f = open('./paper_data.json')
# f = open("./tag_label.json")
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Iterating through the json
# list
papers = []
title_dicts = []
for i in data:
    #print(i.keys())
    #print(i['csl']['title'])
    #print(i['s2data']['referenceCount'],i['s2data']['citationCount'],i['s2data']['influentialCitationCount'])
    #print(len(i['s2data']['references']))
    if 'csl' in i.keys() and i['csl'] != None:
        #print(i['csl'])
        t = i['csl']['title']
        doi = i['doi']
        mini_dict = {"doi":doi, "title":t, "tags":[], "modes":[]}
        title_dicts.append(mini_dict)
  
# Closing file
f.close()

In [15]:
len(title_dicts)

63

In [24]:
titles_json = json.dumps(title_dicts)


In [25]:
with open("tag_label_v2.json", "w") as outfile:
    outfile.write(titles_json)

### To Dos

- add tags to paper tiles
- create bar chart based on tags (incorporate theory/tools distinction)
- (find a more robust way to parse titles)
- collect top infleuntial citations for each paper (from semantic scholar)
- aggregate top papers for each 'group' - as defined by bar chart
- create heatmap based on top paper data
- publish to website

In [ ]:
#### making a bar chart with the paper tags

In [124]:
# read in the json file
f = open('tag_label_v2.json')
paper_tags = json.load(f)

df = pd.DataFrame.from_dict(paper_tags)
df['tags'] = df['tags'].str.join(',')
df['modes'] = df['modes'].str.join('[]')

df.head()

In [126]:
# split the tags column into multiple columns with single tags in them
df[['tags_1', 'tags_2', 'tags_3', 'tags_4']] = df['tags'].apply(lambda x : pd.Series(x.split(',')))
df["idx"] = df.index
df.head()

,doi,title,tags,modes,tags_1,tags_2,tags_3,tags_4,idx
0,10.48550/arXiv.2205.02007,A Computational Inflection for Scientific Disc...,augmented_lit_search,theory,augmented_lit_search,NaN,NaN,NaN,0
1,10.23915/distill.00028,Communicating with Interactive Articles,"augmented_visualization,augmented_formatting",theory,augmented_visualization,augmented_formatting,NaN,NaN,1
2,10.1111/cgf.13720,Capture &amp; Analysis of Active Reading Behav...,"authoring,augmented_visualization,augmented_fo...",tools,authoring,augmented_visualization,augmented_formatting,analytics,2
3,10.1093/comjnl/27.2.97,Literate Programming,"augmented_formatting,authoring",tools,augmented_formatting,authoring,NaN,NaN,3
4,10.1145/3173574.3173606,Exploration and Explanation in Computational N...,"authoring,augmented_formatting",theory,authoring,augmented_formatting,NaN,NaN,4


In [127]:
# format long
df_long = pd.melt(df, id_vars='title' , value_vars=['modes','tags_1', 'tags_2', 'tags_3', 'tags_4'])
df_long.size


945

In [129]:
# combine some tags - for now - ask LE about later
df_long = df_long.replace('augmented_formatting','layout')
df_long = df_long.replace('augmented_visualization','visualization')
df_long = df_long.replace('augmented_lit_search','lit_search')

In [130]:
# hold shift to select multiple
pts = alt.selection(type="multi", encodings=['x'])

# single click if this is easier for creating the heatmap
# pts = alt.selection(type="single", encodings=['x'])


alt.Chart(df_long.dropna()).mark_bar().encode(
    x='value',
    y='count()',
    color=alt.condition(pts, alt.ColorValue("steelblue"), alt.ColorValue("grey"))
).add_selection(pts)

alt.Chart(...)

In [131]:
# try to do a stacked histogram for tools vs theory...
df_long = pd.melt(df, id_vars=['title','modes'] , value_vars=['tags_1', 'tags_2', 'tags_3', 'tags_4'])

# combine some tags - for now - ask LE about later
df_long = df_long.replace('augmented_formatting','layout')
df_long = df_long.replace('augmented_visualization','visualization')
df_long = df_long.replace('augmented_lit_search','lit_search')

In [139]:
source = df_long

pts = alt.selection(type="multi", encodings=['x','color'])

alt.Chart(source.dropna()).mark_bar().encode(
    x='value',
    y='count()',
    color=alt.condition(pts, 'modes', alt.value('gray'))
).add_selection(pts)

alt.Chart(...)

In [146]:
df_long.to_pickle('papers_plus_tags_long_df.pkl')

In [147]:
# new df
df2 = pd.read_pickle('reference_df_with_tags.pkl')
df2.head()

,class_paper,s2_paperId,ref_title,citationCount,influentialCitationCount,year,title,modes,variable,value
0,A Computational Inflection for Scientific Disc...,449e58a29a3971d4d54d9bb28df3b31c60d20483,ACCoRD: A Multi-Document Approach to Generatin...,2,0,2022.0,A Computational Inflection for Scientific Disc...,theory,tags_1,lit_search
1,A Computational Inflection for Scientific Disc...,449e58a29a3971d4d54d9bb28df3b31c60d20483,ACCoRD: A Multi-Document Approach to Generatin...,2,0,2022.0,A Computational Inflection for Scientific Disc...,theory,tags_2,NaN
2,A Computational Inflection for Scientific Disc...,449e58a29a3971d4d54d9bb28df3b31c60d20483,ACCoRD: A Multi-Document Approach to Generatin...,2,0,2022.0,A Computational Inflection for Scientific Disc...,theory,tags_3,NaN
3,A Computational Inflection for Scientific Disc...,449e58a29a3971d4d54d9bb28df3b31c60d20483,ACCoRD: A Multi-Document Approach to Generatin...,2,0,2022.0,A Computational Inflection for Scientific Disc...,theory,tags_4,NaN
4,A Computational Inflection for Scientific Disc...,1b72a2842c535502f639e6082562723717582046,A very preliminary analysis of DALL-E 2,10,0,2022.0,A Computational Inflection for Scientific Disc...,theory,tags_1,lit_search


In [151]:
source_bar

,class_paper,s2_paperId,ref_title,citationCount,influentialCitationCount,year
0,A Computational Inflection for Scientific Disc...,449e58a29a3971d4d54d9bb28df3b31c60d20483,ACCoRD: A Multi-Document Approach to Generatin...,2,0,2022.0
4,A Computational Inflection for Scientific Disc...,1b72a2842c535502f639e6082562723717582046,A very preliminary analysis of DALL-E 2,10,0,2022.0
8,A Computational Inflection for Scientific Disc...,c57293882b2561e1ba03017902df9fc2f289dea2,Hierarchical Text-Conditional Image Generation...,267,39,2022.0
12,A Computational Inflection for Scientific Disc...,a83cdcc0135c58fddf89fc72f1b92b7a9d1e170f,LinkBERT: Pretraining Language Models with Doc...,17,8,2022.0
16,A Computational Inflection for Scientific Disc...,af38697d8acf289ce8b0ebe2bcc638aff0eb1a1d,The Inefficiency of Language Models in Scholar...,1,0,2022.0
...,...,...,...,...,...,...
12864,Going digital,,A multi-tiered approach to hypertext integrati...,0,0,0.0
12868,Going digital,,Control Through Communication,0,0,0.0
12872,Going digital,8e9dd0390cef14c48899f98176b1b8cd3ea3020c,Work-oriented design of computer artifacts,1345,87,1989.0
12876,Going digital,,©ACM00020782,0,0,0.0


In [152]:
source = df2
source_bar = df2.iloc[:,-4:].drop_duplicates()
# alt.data_transformers.disable_max_rows()

pts = alt.selection(type="multi", encodings=['x','color'])

alt.Chart(source_bar.dropna()).mark_bar().encode(
    x='value',
    y='count()',
    color=alt.condition(pts, 'modes', alt.value('gray'))
).add_selection(pts)

alt.Chart(...)

In [ ]:
# other stuff I was workshopping - plotly sunburst plot

In [92]:
df_long = pd.melt(df, id_vars=['title','modes'] , value_vars=['tags_1', 'tags_2', 'tags_3', 'tags_4'])
df_long

,title,modes,variable,value
0,A Computational Inflection for Scientific Disc...,theory,tags_1,augmented_lit_search
1,Communicating with Interactive Articles,theory,tags_1,augmented_visualization
2,Capture &amp; Analysis of Active Reading Behav...,tools,tags_1,authoring
3,Literate Programming,tools,tags_1,augmented_formatting
4,Exploration and Explanation in Computational N...,theory,tags_1,authoring
...,...,...,...,...
247,"Open Science: One Term, Five Schools of Thought",theory,tags_4,NaN
248,The state of OA: a large-scale analysis of the...,theory,tags_4,NaN
249,Going digital,theory,tags_4,NaN
250,ArXiv at 20,theory,tags_4,NaN


In [102]:
df_long_tools = df_long[df_long.modes.isin(['tools'])]
df_long_theory = df_long[df_long.modes.isin(['theory'])]

tools_df = pd.DataFrame(df_long_tools['value'].value_counts().rename_axis('tags').reset_index(name='counts'))
theory_df = pd.DataFrame(df_long_theory['value'].value_counts().rename_axis('tags').reset_index(name='counts'))
tools_df['modes'] = 'tools'
theory_df['modes'] = 'theory'

df_plot = tools_df.merge(theory_df,how='outer')



In [108]:
df_plot = df_plot.replace('augmented_formatting','layout')
df_plot = df_plot.replace('augmented_visualization','visualization')
df_plot = df_plot.replace('augmented_lit_search','lit_search')

In [111]:
import plotly.express as px

# df_plot = df_long

# df_plot = px.data.gapminder().query("year == 2007")
fig = px.sunburst(df_plot, path=['modes', 'tags'], values='counts',
                  hover_data=[])
fig.show()

In [23]:
import altair as alt
from vega_datasets import data

source = data.movies.url

pts = alt.selection(type="single", encodings=['x'])

rect = alt.Chart(data.movies.url).mark_rect().encode(
    alt.X('IMDB_Rating:Q', bin=True),
    alt.Y('Rotten_Tomatoes_Rating:Q', bin=True),
    alt.Color('count()',
        scale=alt.Scale(scheme='greenblue'),
        legend=alt.Legend(title='Total Records')
    )
)

circ = rect.mark_point().encode(
    alt.ColorValue('grey'),
    alt.Size('count()',
        legend=alt.Legend(title='Records in Selection')
    )
).transform_filter(
    pts
)

bar = alt.Chart(source).mark_bar().encode(
    x='Major_Genre:N',
    y='count()',
    color=alt.condition(pts, alt.ColorValue("steelblue"), alt.ColorValue("grey"))
).properties(
    width=550,
    height=200
).add_selection(pts)

alt.vconcat(
    rect + circ,
    bar
).resolve_legend(
    color="independent",
    size="independent"
)

alt.VConcatChart(...)